# 🛢️ PIPELINE LEAK DETECTION v4.0 - CRUDE OIL EDITION

**Enhanced Leak Detection - Optimized for Crude Oil**

✅ **Adjusted for crude oil properties (density, viscosity)**  
✅ **Enhanced elevation correction for denser fluid**  
✅ **Training sekali, pakai berkali-kali**  
✅ **Auto-adjust untuk sensor berapapun (3, 4, 9+)**  
✅ **Tinggal isi NORMAL_PRESSURE, DROP_PRESSURE, SENSOR_LOCATIONS**  

---

## 🛢️ CRUDE OIL vs CONDENSATE:

| Property | Condensate | **Crude Oil** |
|----------|-----------|---------------|
| Specific Gravity | 0.7-0.8 | **0.8-0.95** |
| Viscosity | 0.3-1.0 cP | **5-50+ cP** |
| Elevation Effect | Lower | **Higher** |
| Friction Loss | Lower | **Higher** |

---

## 📋 Workflow:

### 🏗️ PART A: TRAINING (Jalankan Sekali Aja)
- **Cell 1-6**: Setup dan training model
- **Output**: File `leak_detector_crude_oil.sav`

### 🚀 PART B: PREDICTION (Plug & Play!)
- **Cell 7**: Edit data sensor lu, langsung run!
- Bisa 3, 4, 9+ sensor - bebas!

---

## 🛢️ CRUDE OIL PARAMETERS SUMMARY

**This notebook has been optimized for CRUDE OIL with the following adjustments:**

### 1. Fluid Properties:
```python
FLUID_DENSITY = 0.85      # Light crude oil (vs 0.75 for condensate)
FLUID_VISCOSITY = 10      # cP (vs 0.5 for condensate)
```

### 2. Detection Weights (Optimized for crude):
```python
FINAL_ESTIMATE_WEIGHTS = {
    'suspicion': 0.25,      # ↓ from 0.30
    'interpolation': 0.20,  # ↓ from 0.25
    'gradient': 0.20,       # ↑ from 0.15 (viscous fluid)
    'elevation': 0.30,      # ↑ from 0.25 (dense fluid)
    'weighted': 0.05
}
```

### 3. Upstream Bias (Adjusted):
```python
UPSTREAM_BIAS_PRIMARY = -2.2    # vs -2.0 for condensate
UPSTREAM_BIAS_GRADIENT = -1.8   # vs -1.5 for condensate
```

### 💡 Tips:
- **Light crude (API 30-40°)**: Use FLUID_DENSITY = 0.85, VISCOSITY = 5-15 cP
- **Medium crude (API 22-30°)**: Use FLUID_DENSITY = 0.90, VISCOSITY = 15-50 cP
- **Heavy crude (API <22°)**: Use FLUID_DENSITY = 0.93, VISCOSITY = 50-200 cP
- **Calibration**: Jika ada data needle valve test, re-tune UPSTREAM_BIAS untuk akurasi optimal

---

## 📦 STEP 1: IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
import pickle
import warnings
warnings.filterwarnings('ignore')

# Plot configuration
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Libraries loaded successfully!")
print("✅ Pipeline Leak Detection v4.0 - Plug and Play Edition")
print("✅ Ready for training and prediction")

✅ Libraries loaded successfully!
✅ Pipeline Leak Detection v4.0 - Plug and Play Edition
✅ Ready for training and prediction


In [3]:
class EnhancedLeakAnalyzer:
    """
    Enhanced Leak Detection v4.0
    Auto-adjust untuk jumlah sensor berapapun
    """
    
    def __init__(self, base_config, elevation_df=None):
        self.base_config = base_config
        self.elevation_df = elevation_df
        self.has_elevation_data = elevation_df is not None
    
    def predict(self, sensor_locations, normal_pressure, drop_pressure, sensor_names=None):
        """
        MAIN PREDICTION FUNCTION
        
        Input:
            sensor_locations: array lokasi sensor (KP in km) - bisa 3, 4, 9+ sensor
            normal_pressure: array tekanan normal (psi)
            drop_pressure: array tekanan drop (psi)
            sensor_names: list nama sensor (optional)
        
        Output:
            dict dengan hasil analisis lengkap
        """
        # Convert to arrays
        locations = np.array(sensor_locations)
        normal_p = np.array(normal_pressure)
        drop_p = np.array(drop_pressure)
        n_sensors = len(locations)
        
        # Validate
        if len(normal_p) != n_sensors:
            raise ValueError(f"normal_pressure harus {n_sensors} elemen (sesuai jumlah sensor)")
        if len(drop_p) != n_sensors:
            raise ValueError(f"drop_pressure harus {n_sensors} elemen (sesuai jumlah sensor)")
        
        # Generate sensor names if not provided
        if sensor_names is None:
            sensor_names = [f'Sensor {i+1} (KP {loc:.1f})' for i, loc in enumerate(locations)]
        
        # Get elevations for these sensor locations
        if self.has_elevation_data:
            elev_interp = interpolate.interp1d(
                self.elevation_df['distance_km'],
                self.elevation_df['elevation'],
                kind='cubic',
                fill_value='extrapolate'
            )
            elevations = elev_interp(locations)
        else:
            elevations = np.zeros(n_sensors)
        
        # Calculate metrics
        delta_p = normal_p - drop_p
        abs_delta_p = np.abs(delta_p)
        
        with np.errstate(divide='ignore', invalid='ignore'):
            pressure_ratio = abs_delta_p / np.abs(normal_p) * 100
        pressure_ratio = np.nan_to_num(pressure_ratio, 0.0)
        
        # Suspicion index
        suspicion_index = self._calculate_suspicion_index(abs_delta_p, pressure_ratio, n_sensors)
        
        # All detection methods
        susp_loc = self._suspicion_method(locations, suspicion_index)
        grad_loc = self._gradient_method(locations, normal_p, drop_p)
        interp_loc = self._interpolation_method(locations, abs_delta_p)
        weighted_loc = self._weighted_method(locations, suspicion_index)
        elev_loc = self._elevation_method(locations, normal_p, drop_p, elevations, n_sensors)
        
        # Weighted final estimate
        cfg = self.base_config
        final_estimate = (
            susp_loc * cfg['FINAL_ESTIMATE_WEIGHTS']['suspicion'] +
            interp_loc * cfg['FINAL_ESTIMATE_WEIGHTS']['interpolation'] +
            grad_loc * cfg['FINAL_ESTIMATE_WEIGHTS']['gradient'] +
            elev_loc * cfg['FINAL_ESTIMATE_WEIGHTS']['elevation'] +
            weighted_loc * cfg['FINAL_ESTIMATE_WEIGHTS']['weighted']
        )
        
        estimate_std = np.std([susp_loc, interp_loc, grad_loc, elev_loc, weighted_loc])
        
        # Confidence
        if estimate_std < 2:
            confidence = "VERY HIGH (95%+)"
        elif estimate_std < 4:
            confidence = "HIGH (90-95%)"
        elif estimate_std < 6:
            confidence = "HIGH (85-90%)"
        else:
            confidence = "MEDIUM (75-85%)"
        
        # Return results
        return {
            'final_estimate': final_estimate,
            'estimate_std': estimate_std,
            'confidence': confidence,
            'zones': {
                'focus': (final_estimate - 3, final_estimate + 3),
                'critical': (final_estimate - 5, final_estimate + 5),
                'primary': (final_estimate - 10, final_estimate + 10)
            },
            'top_sensor_idx': np.argmax(suspicion_index),
            'methods': {
                'suspicion': susp_loc,
                'interpolation': interp_loc,
                'gradient': grad_loc,
                'elevation': elev_loc,
                'weighted': weighted_loc
            },
            'sensor_data': {
                'locations': locations,
                'names': sensor_names,
                'elevations': elevations,
                'normal_pressure': normal_p,
                'drop_pressure': drop_p,
                'delta_pressure': delta_p,
                'abs_delta_pressure': abs_delta_p,
                'pressure_ratio': pressure_ratio,
                'suspicion_index': suspicion_index
            }
        }
    
    def _calculate_suspicion_index(self, abs_delta_p, pressure_ratio, n_sensors):
        cfg = self.base_config
        suspicion_index = np.zeros(n_sensors)
        
        for i in range(n_sensors):
            delta_factor = abs_delta_p[i]
            ratio_factor = pressure_ratio[i]
            
            if i > 0 and i < n_sensors - 1:
                neighbor_avg = (abs_delta_p[i-1] + abs_delta_p[i+1]) / 2
                neighbor_diff = abs_delta_p[i] - neighbor_avg
            elif i == 0 and n_sensors > 1:
                neighbor_diff = abs_delta_p[i] - abs_delta_p[i+1]
            elif i == n_sensors - 1 and n_sensors > 1:
                neighbor_diff = abs_delta_p[i] - abs_delta_p[i-1]
            else:
                neighbor_diff = 0
            
            neighbor_factor = max(0, neighbor_diff)
            
            suspicion_index[i] = (
                delta_factor * cfg['SUSPICION_WEIGHTS'][0] +
                ratio_factor * cfg['SUSPICION_WEIGHTS'][1] +
                neighbor_factor * cfg['SUSPICION_WEIGHTS'][2]
            )
        
        return suspicion_index
    
    def _suspicion_method(self, locations, suspicion_index):
        cfg = self.base_config
        top_idx = np.argmax(suspicion_index)
        location = locations[top_idx] + cfg['UPSTREAM_BIAS_PRIMARY']
        return max(location, locations[0])
    
    def _gradient_method(self, locations, normal_p, drop_p):
        cfg = self.base_config
        if len(locations) < 2:
            return locations[0]
        
        changes = []
        locs = []
        for i in range(len(locations) - 1):
            dist = locations[i+1] - locations[i]
            if dist > 0:
                norm_grad = (normal_p[i+1] - normal_p[i]) / dist
                drop_grad = (drop_p[i+1] - drop_p[i]) / dist
                changes.append(np.abs(norm_grad - drop_grad))
                locs.append((locations[i] + locations[i+1]) / 2)
        
        if not changes:
            return locations[0]
        
        max_idx = np.argmax(changes)
        return locs[max_idx] + cfg['UPSTREAM_BIAS_GRADIENT']
    
    def _interpolation_method(self, locations, abs_delta_p):
        cfg = self.base_config
        if len(locations) < 4:
            max_idx = np.argmax(abs_delta_p)
            return locations[max_idx] + cfg['UPSTREAM_BIAS_INTERP']
        
        try:
            f = interpolate.interp1d(locations, abs_delta_p, kind='cubic', fill_value='extrapolate')
            x_fine = np.linspace(locations.min(), locations.max(), 2000)
            y_fine = f(x_fine)
            peak_idx = np.argmax(y_fine)
            return x_fine[peak_idx] + cfg['UPSTREAM_BIAS_INTERP']
        except:
            max_idx = np.argmax(abs_delta_p)
            return locations[max_idx] + cfg['UPSTREAM_BIAS_INTERP']
    
    def _weighted_method(self, locations, suspicion_index):
        cfg = self.base_config
        total = np.sum(suspicion_index)
        if total == 0:
            return np.mean(locations)
        weighted_loc = np.sum(suspicion_index * locations) / total
        return weighted_loc + cfg['UPSTREAM_BIAS_WEIGHTED']
    
    def _elevation_method(self, locations, normal_p, drop_p, elevations, n_sensors):
        cfg = self.base_config
        
        if not self.has_elevation_data:
            return locations[np.argmax(np.abs(normal_p - drop_p))] + cfg['UPSTREAM_BIAS_PRIMARY']
        
        # Elevation-corrected pressure
        psi_per_meter = cfg['PSI_PER_METER'] * cfg['FLUID_DENSITY']
        ref_elev = elevations[0]
        elev_corr = (elevations - ref_elev) * psi_per_meter
        
        normal_corr = normal_p - elev_corr
        drop_corr = drop_p - elev_corr
        
        # Hydraulic anomaly
        anomaly_scores = np.zeros(n_sensors)
        for i in range(1, n_sensors):
            dist = locations[i] - locations[i-1]
            if dist > 0:
                exp_grad = (normal_corr[i-1] - normal_corr[i]) / dist
                act_grad = (drop_corr[i-1] - drop_corr[i]) / dist
                anom = abs(act_grad - exp_grad)
                anomaly_scores[i-1] += anom * 0.5
                anomaly_scores[i] += anom * 0.5
        
        max_idx = np.argmax(anomaly_scores)
        return locations[max_idx] + cfg['UPSTREAM_BIAS_PRIMARY']

print("✅ EnhancedLeakAnalyzer class loaded!")

✅ EnhancedLeakAnalyzer class loaded!


---

# ⚙️ STEP 2: TRAINING CONFIGURATION (CRUDE OIL)

**⚠️ Edit bagian ini untuk TRAINING model (run sekali aja)**

## 🛢️ CRUDE OIL ADJUSTMENTS:

1. **FLUID_DENSITY = 0.85** (Light crude)  
   - Light crude: 0.85 (API ~35°)  
   - Medium crude: 0.90 (API ~25°)  
   - Heavy crude: 0.93 (API ~20°)  

2. **FLUID_VISCOSITY = 10 cP**  
   - Light: 5-15 cP  
   - Medium: 15-50 cP  
   - Heavy: >50 cP  

3. **ELEVATION WEIGHT = 0.30** (increased from 0.25)  
   - Crude oil lebih dense → stronger elevation effect  

4. **GRADIENT WEIGHT = 0.20** (increased from 0.15)  
   - Viscous fluid → clearer pressure gradient pattern  

5. **UPSTREAM BIAS** slightly adjusted for crude oil behavior  

---

Ini data untuk **build model** pertama kali. Setelah model tersimpan,  
lu gak perlu edit cell ini lagi.

---
---
---

# 🚀 STEP 7: PLUG AND PLAY PREDICTION ⭐

## ✏️ EDIT BAGIAN INI SESUAI DATA LU:

**Yang perlu lu isi:**
1. **SENSOR_LOCATIONS** - Lokasi sensor (KP dalam km)
2. **NORMAL_PRESSURE** - Tekanan normal (psi)
3. **DROP_PRESSURE** - Tekanan saat anomaly (psi)
4. **SENSOR_NAMES** (optional) - Nama sensor

**Jumlah elemen harus sama!**
- 3 sensor → 3 locations, 3 normal, 3 drop
- 4 sensor → 4 locations, 4 normal, 4 drop
- 9 sensor → 9 locations, 9 normal, 9 drop

---

In [4]:
# ============================================================================
# 📝 EDIT BAGIAN INI - DATA SENSOR LU
# ============================================================================

# Contoh 1: Full 9 sensors
#SENSOR_LOCATIONS = [5.0, 18.2, 34.2, 45.0, 59.0, 65.5, 75.0, 86.0, 98.0]
#NORMAL_PRESSURE = [219.69, 175.06, 140.35, 125.02, 107.81, 85.53, 71.34, 55.86, 34.57]
#DROP_PRESSURE = [213.87, 169.22, 133.75, 118.62, 100.61, 79.87, 66.70, 52.14, 32.51]

# Contoh 2: Hanya 4 sensors (uncomment untuk pakai)
SENSOR_LOCATIONS = [0, 7.14]
NORMAL_PRESSURE = [138.8, 111.7]
DROP_PRESSURE = [131.6, 108]

# Contoh 3: Hanya 3 sensors (uncomment untuk pakai)
# SENSOR_LOCATIONS = [5.0, 59.0, 98.0]
# NORMAL_PRESSURE = [219.69, 107.81, 34.57]
# DROP_PRESSURE = [213.87, 100.61, 32.51]

# Nama sensor (optional - kalau gak diisi, auto-generate)
SENSOR_NAMES = None  # Set ke None untuk auto-generate
# Atau isi manual:
# SENSOR_NAMES = ['Sensor A', 'Sensor B', 'Sensor C', ...]

# ============================================================================
# 🤖 AUTO PREDICTION - Jangan edit di bawah ini!
# ============================================================================

print("="*80)
print("🚀 PLUG AND PLAY PREDICTION")
print("="*80)

# Load model
with open('bjg_model.sav', 'rb') as f:
    model = pickle.load(f)

print(f"\n✅ Model loaded!")
print(f"   Analyzing {len(SENSOR_LOCATIONS)} sensors...")
print(f"   Locations: KP {min(SENSOR_LOCATIONS):.1f} - KP {max(SENSOR_LOCATIONS):.1f}")

# Validate
if len(NORMAL_PRESSURE) != len(SENSOR_LOCATIONS):
    print(f"\n❌ ERROR: NORMAL_PRESSURE ({len(NORMAL_PRESSURE)}) != SENSOR_LOCATIONS ({len(SENSOR_LOCATIONS)})")
    print("   Jumlah elemen harus sama!")
elif len(DROP_PRESSURE) != len(SENSOR_LOCATIONS):
    print(f"\n❌ ERROR: DROP_PRESSURE ({len(DROP_PRESSURE)}) != SENSOR_LOCATIONS ({len(SENSOR_LOCATIONS)})")
    print("   Jumlah elemen harus sama!")
else:
    # Predict
    results = model.predict(SENSOR_LOCATIONS, NORMAL_PRESSURE, DROP_PRESSURE, SENSOR_NAMES)
    
    # Display Results
    print("\n" + "="*80)
    print("🎯 LEAK DETECTION RESULTS")
    print("="*80)
    
    print(f"\n>>> ESTIMATED LEAK LOCATION: KP {results['final_estimate']:.2f} km <<<\n")
    print(f"Confidence: {results['confidence']}")
    print(f"Uncertainty: ±{results['estimate_std']:.2f} km")
    
    print("\n" + "="*80)
    print("🎯 INSPECTION ZONES (PRIORITIZED)")
    print("="*80)
    
    focus = results['zones']['focus']
    critical = results['zones']['critical']
    primary = results['zones']['primary']
    
    print(f"\n1. 🔴 FOCUS ZONE (HIGHEST PRIORITY)")
    print(f"   KP {focus[0]:.1f} to {focus[1]:.1f} (6 km area)")
    print(f"   → Start here!")
    
    print(f"\n2. 🟠 CRITICAL ZONE")
    print(f"   KP {critical[0]:.1f} to {critical[1]:.1f} (10 km area)")
    
    print(f"\n3. 🟡 PRIMARY ZONE")
    print(f"   KP {primary[0]:.1f} to {primary[1]:.1f} (20 km area)")
    
    print("\n" + "="*80)
    print("⚠️  MOST SUSPICIOUS SENSOR")
    print("="*80)
    
    top_idx = results['top_sensor_idx']
    sensor_data = results['sensor_data']
    print(f"\n{sensor_data['names'][top_idx]}")
    print(f"Location: KP {sensor_data['locations'][top_idx]:.1f}")
    print(f"Suspicion Index: {sensor_data['suspicion_index'][top_idx]:.2f}")
    
    print("\n" + "="*80)
    print("📋 METHOD BREAKDOWN")
    print("="*80)
    
    methods = results['methods']
    print(f"\n  Suspicion Index:     KP {methods['suspicion']:.2f}")
    print(f"  Interpolation:       KP {methods['interpolation']:.2f}")
    print(f"  Gradient:            KP {methods['gradient']:.2f}")
    print(f"  Elevation/Hydraulic: KP {methods['elevation']:.2f}")
    print(f"  Weighted Average:    KP {methods['weighted']:.2f}")
    
    print("\n" + "="*80)
    print("📊 SENSOR DETAILS")
    print("="*80)
    
    print(f"\n{'Sensor':<30} {'KP':>6} {'Elev':>7} {'Normal':>8} {'Drop':>8} {'ΔP':>7} {'Score':>6}")
    print("-" * 95)
    
    for i in range(len(sensor_data['names'])):
        elev_str = f"{sensor_data['elevations'][i]:.1f}" if model.has_elevation_data else "N/A"
        print(f"{sensor_data['names'][i]:<30} "
              f"{sensor_data['locations'][i]:>6.1f} "
              f"{elev_str:>7} "
              f"{sensor_data['normal_pressure'][i]:>8.2f} "
              f"{sensor_data['drop_pressure'][i]:>8.2f} "
              f"{sensor_data['delta_pressure'][i]:>7.2f} "
              f"{sensor_data['suspicion_index'][i]:>6.2f}")
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE!")
    print("="*80)

🚀 PLUG AND PLAY PREDICTION

✅ Model loaded!
   Analyzing 2 sensors...
   Locations: KP 0.0 - KP 7.1

🎯 LEAK DETECTION RESULTS

>>> ESTIMATED LEAK LOCATION: KP -0.68 km <<<

Confidence: VERY HIGH (95%+)
Uncertainty: ±1.51 km

🎯 INSPECTION ZONES (PRIORITIZED)

1. 🔴 FOCUS ZONE (HIGHEST PRIORITY)
   KP -3.7 to 2.3 (6 km area)
   → Start here!

2. 🟠 CRITICAL ZONE
   KP -5.7 to 4.3 (10 km area)

3. 🟡 PRIMARY ZONE
   KP -10.7 to 9.3 (20 km area)

⚠️  MOST SUSPICIOUS SENSOR

Sensor 1 (KP 0.0)
Location: KP 0.0
Suspicion Index: 5.77

📋 METHOD BREAKDOWN

  Suspicion Index:     KP 0.00
  Interpolation:       KP -2.00
  Gradient:            KP 1.77
  Elevation/Hydraulic: KP -2.20
  Weighted Average:    KP 0.46

📊 SENSOR DETAILS

Sensor                             KP    Elev   Normal     Drop      ΔP  Score
-----------------------------------------------------------------------------------------------
Sensor 1 (KP 0.0)                 0.0    55.8   138.80   131.60    7.20   5.77
Sensor 2 (KP 7.1)   